In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

colorscheme = px.colors.sequential.Viridis[::3]
symbols = ['circle', 'square', 'x', 'diamond', 'triangle-up']

df = pd.read_csv('results.csv')
df = df[df['trial name'] == 'house720-grey-h1-3iter-data']
df

# estimated service time
estimated_service_time = 8629 #720

In [ ]:
mean = df.groupby(["type", "threads"]).mean().reset_index().sort_values(by=["type", "threads"])
std = df.groupby(["type", "threads"]).std().reset_index().sort_values(by=["type", "threads"])

means = {name:d for name, d in mean.groupby(["type"])}
std = {name:d for name, d in std.groupby(["type"])}
types = means.keys()
threads = pd.unique(mean["threads"]).tolist()

def get_fig(y, title, xtitle, ytitle, log_x=True, log_y=True):
    types = means.keys()
    threads = pd.unique(mean["threads"]).tolist()
    items = means["seq"]["items"]

    fig = go.Figure()

    for type in types:
        if type == "seq":
            continue
        fig.add_trace(go.Scatter(x=threads, y=means[type][y], name=type, error_y=dict(type="data", array=std[type][y])))

     # Add ideal metric
    if y == "completion time":
        seq_completion_time = means["seq"]["completion time"].iloc[0]
        ideal_completion_time = [seq_completion_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=ideal_completion_time, name="ideal", line=dict(dash="dash")))

        estimated = [estimated_service_time * items / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=estimated, name="lower bound", line=dict(dash="dash")))
    elif y == "service time":
        seq_service_time = means["seq"]["service time"].iloc[0]
        ideal_service_time = [seq_service_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=ideal_service_time, name="ideal", line=dict(dash="dash")))

        estimated = [estimated_service_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=estimated, name="lower bound", line=dict(dash="dash")))
    elif y == "efficiency":
        ones = pd.Series(np.ones(len(threads)))
        fig.add_trace(go.Scatter(x=threads, y=ones, name="ideal", line=dict(dash="dash")))
    else:
        fig.add_trace(go.Scatter(x=threads, y=threads, name="ideal", line=dict(dash="dash")))

    for i, trace in enumerate(fig.data):
        # use different symbol and color for each trace
        trace.marker.symbol = symbols[i]
        trace.line.color = colorscheme[i]
        # trace.line.color = "black"

    fig.update_layout(
        title=title,
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        xaxis = dict(
            tickmode = 'array',
            tickvals = threads,
        ),
    )

    if log_x:
        fig.update_xaxes(type="log")

    if log_y:
        fig.update_yaxes(type="log")

    return fig

In [ ]:
get_fig(y="speedup", title="Speedup vs. Number of threads", xtitle="# Threads", ytitle="Speedup")

In [ ]:
get_fig(y="completion time", title="Completion time vs Number of threads", xtitle="# Threads", ytitle="Time (µs)")

In [ ]:
get_fig(y="service time", title="Service time vs Number of threads", xtitle="# Threads", ytitle="Time (µs)")

In [ ]:
get_fig(y="scalability", title="Scalability vs. Number of threads", xtitle="# Threads", ytitle="Scalability")

In [ ]:
get_fig(y="efficiency", title="Efficiency vs. Number of threads", xtitle="# Threads", ytitle="Efficiency")